In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# Replace 'your_file.csv' with the path to your CSV file
data = pd.read_csv('train.csv')
data.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# delete columns with more than 50% NaN
threshold = 0.5
data_filtered1 = data.loc[:, data.isnull().mean() < threshold]
data_filtered1.columns.tolist()

['id',
 'Basic_Demos-Enroll_Season',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-Season',
 'CGAS-CGAS_Score',
 'Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-Season',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-Season',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'PCIAT-Season',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PC

In [4]:
# delete rows with more than 50% NaN
threshold = 0.5
data_filtered2 = data_filtered1.loc[data.notnull().mean(axis=1) >= threshold, :]
data_filtered2

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,8,0,NaN,NaN,Fall,17.139810,52.5,67.2,...,2.0,2.0,1.0,22.0,Fall,41.0,58.0,Fall,2.0,0.0
3954,ff759544,Summer,7,1,NaN,NaN,Summer,13.927006,48.5,46.6,...,3.0,3.0,0.0,33.0,Summer,48.0,67.0,Summer,0.0,1.0
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0


In [5]:
#Delete repetitive columns like Height or Weight since we have BMI and delete seasons & SDS-SDS, which I don't think that useful. And then fill empty BMI by average based on sex
data_drop = data_filtered2.drop(columns=['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Height', 'Physical-Weight', 'Physical-Season', 'FGC-Season', 'BIA-Season', 'PCIAT-Season','SDS-Season', 'SDS-SDS_Total_Raw','PreInt_EduHx-Season',
 'SDS-SDS_Total_T',])

# Calculate the average BMI for each gender
avg_bmi_by_sex = data_drop.groupby('Basic_Demos-Sex')['Physical-BMI'].mean()

# Fill missing BMI values based on the average BMI of the same gender
data_drop['Physical-BMI'] = data_drop.apply(
    lambda row: avg_bmi_by_sex[row['Basic_Demos-Sex']] if pd.isnull(row['Physical-BMI']) else row['Physical-BMI'],
    axis=1
)
data_drop[data_drop['Physical-BMI'].isna() == True]

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_CU_Zone,...,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,PreInt_EduHx-computerinternet_hoursday,sii


In [6]:
# List of columns to fill NaN with their averages
columns_to_fill = ['Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP','PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMI',
 'BIA-BIA_BMC',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'PCIAT-PCIAT_Total',
 'PreInt_EduHx-computerinternet_hoursday',
 'sii']

# Fill NaN values with the mean of each column
data_drop[columns_to_fill] = data_drop[columns_to_fill].apply(lambda col: col.fillna(col.mean()))
print(data_drop[columns_to_fill].isna().sum())

Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
PCIAT-PCIAT_01                            0
PCIAT-PCIAT_02                            0
PCIAT-PCIAT_03                            0
PCIAT-PCIAT_04                            0
PCIAT-PCIAT_05                            0
PCIAT-PCIAT_06                            0
PCIAT-PCIAT_07                            0
PCIAT-PCIAT_08                            0
PCIAT-PCIAT_09                            0
PCIAT-PCIAT_10                            0
PCIAT-PCIAT_11                            0
PCIAT-PCIAT_12                            0
PCIAT-PCIAT_13                            0
PCIAT-PCIAT_14                            0
PCIAT-PCIAT_15                            0
PCIAT-PCIAT_16                            0
PCIAT-PCIAT_17                            0
PCIAT-PCIAT_18                            0
PCIAT-PCIAT_19                            0
PCIAT-PCIAT_20                  

The Parent-Child Internet Addiction Test (PCIAT) is a 20-item questionnaire designed to assess internet addiction behaviors in children from the parents' perspective. It evaluates characteristics such as compulsivity, escapism, and dependency, and examines how these behaviors impact personal, occupational, and social functionin


The Children's Global Assessment Scale (CGAS) is a numeric scale used by mental health clinicians to evaluate the general functioning of children and adolescents aged 4 to 16. It provides a single score ranging from 1 to 100, with higher scores indicating better overall functioning. This tool assesses psychological and social aspects of a young person's daily life, including home, school, and peer interactions.


The Self-Directed Search (SDS) is a career assessment tool that helps individuals identify suitable career paths based on their interests and abilities. It generates various scores to guide career decision-making.

### 1. Usage Duration
Indicators:
Average Daily Usage Time: Total time spent online each day.
Continuous Usage Time: Longest uninterrupted session of internet use.
Importance:
Excessive usage duration is a primary symptom of internet addiction.
Long continuous sessions can indicate a lack of self-control over usage.
Related Data Fields:
FGC_CU (Current Usage) and FGC_PU (Past Usage).


### 2. Usage Frequency
Indicators:
App Launch Frequency: Number of times apps or platforms are accessed daily.
Activity Interaction Count: Number of specific online interactions, such as clicks, scrolling, or social media engagement (likes, comments).
Importance:
Frequent usage and high interaction rates can reflect compulsive behavior.
Indicates reliance on specific apps or platforms, such as social media or gaming.
Related Data Fields:
FGC_SRR (Social Network Usage).


### 3. Time-of-Day Usage Patterns
Indicators:
Night Usage Time: Time spent online during late hours (e.g., after 10:00 PM).
Time Zone Distribution: Behavior patterns across different periods (morning, afternoon, evening).
Importance:
Late-night usage is a strong warning sign of internet addiction, as it disrupts sleep and daily routines.
Time-based patterns reveal how internet use interferes with regular life.
Related Data Fields:
FGC_CU_Zone and FGC_PU_Zone.

### 4. Self-Regulation Level
Indicators:
Planned vs. Unplanned Usage Ratio: Proportion of intentional usage versus impulsive or unplanned usage.
Self-Control Failure Rate: Number of instances where the user attempted but failed to limit their internet use.
Importance:
Evaluates the user's ability to control their behavior, which is often impaired in addiction.
Can inform interventions aimed at improving self-regulation skills.
Related Data Fields:
FGC_SRL and FGC_SRL_Zone.

### 5. Psychological and Behavioral Dependence
Indicators:
Emotion-Driven Usage: Online activity motivated by boredom, anxiety, or stress.
Avoidance Behavior: Using the internet as a way to escape real-life responsibilities or problems.
Importance:
Reveals the underlying psychological drivers of addiction, such as stress relief or emotional avoidance.
Helps design targeted psychological interventions.
Data Collection:
Requires surveys or tools to monitor emotional states during usage.

### 6. Social Network Dependency
Indicators:
Social Interaction Preference: Time spent on online interactions versus real-life interactions.
Virtual Identity Engagement: Effort and time spent managing one’s online presence or identity (e.g., on social media).
Importance:
Highlights dependency on online social interactions, often due to social isolation.
Indicates whether addiction stems from reliance on virtual connections.
Related Data Fields:
FGC_SRR and FGC_SRR_Zone.


In [7]:
len(data_drop.columns)

57

In [8]:
data_withNaN = data_drop.loc[:, data_drop.isna().sum() != 0]
data_withNaN.columns.tolist()

['CGAS-CGAS_Score']

In [9]:
columns_to_select = [
 'CGAS-CGAS_Score',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_TBW']
data_tobeused = data_drop[columns_to_select].copy()
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Select the features to standardize
columns_to_scale = ['BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_TBW']
data_tobeused[columns_to_scale] = scaler.fit_transform(data_tobeused[columns_to_scale])

# Display the standardized data
print(data_tobeused.head())

# data_tobeused
#data_tobeused.loc[:, data.isna().any()].columns.tolist()

/Users/ychen3641/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


   CGAS-CGAS_Score   BIA-BIA_BMI   BIA-BIA_BMR   BIA-BIA_DEE  BIA-BIA_TBW
0             51.0 -5.704082e-01 -1.879150e-01 -2.333122e-01    -0.190507
1              NaN -1.230190e+00 -1.853473e-01 -2.306002e-01    -0.240885
2             71.0  6.597979e-15  1.404100e-16  3.709131e-16     0.000000
3             71.0 -2.418987e-01 -6.506861e-02 -5.736092e-02    -0.071566
5             50.0  2.518825e+00  5.815326e-02 -2.758574e-02     0.081560


In [10]:
data_tobeused.columns.tolist()

['CGAS-CGAS_Score', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_TBW']

In [11]:
X = data_tobeused[['BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_TBW']]
y = data_tobeused['CGAS-CGAS_Score']
X_train = X.loc[y.dropna().index,:]
y_train = y.dropna()
X_test = X[y.isna()]
y_test = y[y.isna()]
print(y_train.isna().sum())
print(y_test.isna().sum())

# Ensure all data in X_train is numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1) 

X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1) 
# Define the model with proper weight initialization
model = nn.Linear(4, 1)
torch.nn.init.xavier_uniform_(model.weight)
torch.nn.init.zeros_(model.bias)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training
num_epochs = 1000
for epoch in range(num_epochs):
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:  # Print loss every 10 epochs
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Predictions
predictions = model(X_test)
print(f"Predictions:\n{predictions}")
print(f"Number of test samples: {X_test.shape[0]}")


0
458
Epoch 10/1000, Loss: 3078.042236328125
Epoch 20/1000, Loss: 2100.1435546875
Epoch 30/1000, Loss: 1447.7889404296875
Epoch 40/1000, Loss: 1012.5606689453125
Epoch 50/1000, Loss: 722.1673583984375
Epoch 60/1000, Loss: 528.3967895507812
Epoch 70/1000, Loss: 399.0893249511719
Epoch 80/1000, Loss: 312.79351806640625
Epoch 90/1000, Loss: 255.19769287109375
Epoch 100/1000, Loss: 216.7538604736328
Epoch 110/1000, Loss: 191.09153747558594
Epoch 120/1000, Loss: 173.95986938476562
Epoch 130/1000, Loss: 162.52206420898438
Epoch 140/1000, Loss: 154.8850555419922
Epoch 150/1000, Loss: 149.78524780273438
Epoch 160/1000, Loss: 146.37933349609375
Epoch 170/1000, Loss: 144.10440063476562
Epoch 180/1000, Loss: 142.58462524414062
Epoch 190/1000, Loss: 141.5691375732422
Epoch 200/1000, Loss: 140.8904266357422
Epoch 210/1000, Loss: 140.4365997314453
Epoch 220/1000, Loss: 140.13302612304688
Epoch 230/1000, Loss: 139.9298095703125
Epoch 240/1000, Loss: 139.79360961914062
Epoch 250/1000, Loss: 139.702209

In [16]:
# Convert predictions to numpy
predictions_np = predictions.detach().numpy().flatten()  # Flatten for compatibility with pandas

# Find the rows where the target column is missing
missing_indices = data_drop[data_tobeused['CGAS-CGAS_Score'].isna()].index

# Fill missing values with predictions
data_drop.loc[missing_indices, 'CGAS-CGAS_Score'] = predictions_np

# Verify the filled column
print(data_drop['CGAS-CGAS_Score'])


0       51.000000
1       66.047066
2       71.000000
3       71.000000
5       50.000000
          ...    
3953    65.425484
3954    66.126434
3955    60.000000
3957    68.000000
3958    70.000000
Name: CGAS-CGAS_Score, Length: 2659, dtype: float64


In [17]:
# Save the DataFrame to a CSV file
data_drop.to_csv('filled_data.csv', index=False)  # index=False excludes the index column